# Loading Libraries

In [2]:
import matplotlib.pyplot as plt
import statsmodels.tsa.seasonal as smt
import statsmodels.tsa.vector_ar.dynamic as dm 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import datetime as dt
from sklearn import linear_model 
from sklearn.metrics import mean_absolute_error
import plotly
import nsepy
from datetime import date
from nsepy import get_history
from tkinter import *
from tkinter import ttk
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
#import tkMessageBox
#import tkFont
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
#from PIL import ImageTk,Image
import matplotlib.pyplot as plt
import statsmodels.tsa.seasonal as smt
import numpy as np # linear algebra
import datetime as dt
from sklearn import linear_model 
from sklearn.metrics import mean_absolute_error
import plotly
from tkinter import *
import tkinter as tk
#root = tk.Tk()
import ast
import time
from statsmodels.tsa.vector_ar.var_model import VAR
import pandas_datareader as web
from nsetools import Nse
#import datetime internal datetime module
#datetime is a Python module
import datetime
import os


companies=os.listdir('/Users/vikramreddy/Desktop/HISTORICAL_DATA')[1:]
os.chdir('/Users/vikramreddy/Desktop/HISTORICAL_DATA')


In [10]:
import pickle
nse = Nse()
words = [word.replace('_data.csv','') for word in companies]
Symbol2Name = pickle.load(open('company_symbol_name_dict.d','rb'))

# Creating Function

In [11]:
def load_data(event):
    #global data
    global file
    global Name
    

    # display element selected on list
    print('(event) previous:', event.widget.get('active'))
    file=event.widget.get(event.widget.curselection())
    for name, age in Symbol2Name.items():
        if name == file:
            Name=name
            x='_data.csv'
            global p

            p=str(name+x)

            print(p)



# Functions for Loading Data using Yahoo API

In [12]:
def read_data():
    global file
    global concat
    global p
    global name
    if file!='':
        df=pd.DataFrame(pd.read_csv(p))
        start = datetime.datetime(2018, 2, 19)
        dt=datetime.datetime.now().strftime('%Y,%m,%d').split(',')
        end=datetime.datetime(int(dt[0]),int(dt[1]),int(dt[2]))

 
    #DataReader method name is case sensitive
        live_file=str(Name+'.NS')
        label1['text']='Status:Loading...'
        global yahoo

        yahoo = web.DataReader(live_file,'yahoo', start, end)
        label1['text']='Status:Loading...'
    #invoke to_csv for df dataframe object from 
    #DataReader method in the pandas_datareader library
 
    #..\first_yahoo_prices_to_csv_demo.csv must not
    #be open in another app, such as Excel
 
        yahoo.to_csv('first_yahoo_prices_volumes_to_csv_demo.csv')
        #yahoo=yahoo.iloc[:len(yahoo)-1,:]
        for i in tree.get_children():
              tree.delete(i)
        if len(df)!=0:
            df.Date=pd.to_datetime(df.Date)
            df=df.set_index('Date')
            df=df.iloc[:,:4]
            yahoo=yahoo.iloc[:,:4]
            yahoo.columns=df.columns
            frames = [df, yahoo]
            concat = pd.concat(frames)
            for i in range(len(concat)-1):
                tree.insert('', 'end', text=concat.index[i],values=(concat.open[i],concat.high[i],concat.low[i],concat.close[i]))
#tree.config(columns = ['Open','high','low','close'])

        label1['text']=str(Name)+' Data loaded from '+ str(concat.index[0].strftime('%Y-%m-%d').split(','))+' to '+str(concat.index[len(concat)-2].strftime('%Y-%m-%d').split(','))
    file=''

# Functions for traning the model

In [13]:
def train_fun():
    global concat
    global file
    
    if file=='':
        #print(concat)
        cols = concat.columns
        for j in cols:
            for i in range(0,len(concat)):
                if concat[j][i] == 'null':
                    concat[j][i] = concat[j][i-1]
        for item in list(concat):
            concat[item]=pd.to_numeric(concat[item])
            #print(concat.dtypes)
        concat_new=concat.iloc[:len(concat)-1,:]
        model = VAR(endog=concat_new)
        results = model.fit(maxlags=30, ic='aic')
        lag_order = results.k_ar

        
        pred=pd.DataFrame(results.forecast(concat_new.values,1))
        pred=round(pred,2)
        pred=pred.transpose()
        pred['live']=concat.values[-1]
        pred['names']=['open','high','low','close']
        concat=round(concat,2)
        pred=round(pred,2)
        if len(pred)!=0:
            for i in tree2.get_children():
                  tree2.delete(i)
            label2['text']=str(concat.index[len(concat)-1].strftime('%Y-%m-%d').split(','))
            label3['text']='        Predicted     VS      Live    '
            for i in range(len(pred)):
                tree2.insert('', 'end', text=pred.names[i],values=(pred.iloc[i,0],pred.live[i]))
        top_gainer=pd.DataFrame(nse.get_top_gainers())
        label8['text']='Top Gainers today [ High price ]'
        label4['text']=str(top_gainer.symbol[0])+'  -  '+str(top_gainer.highPrice[0])
        label5['text']=str(top_gainer.symbol[1])+'  -  '+str(top_gainer.highPrice[1])
        label6['text']=str(top_gainer.symbol[2])+'  -  '+str(top_gainer.highPrice[2])
        label7['text']=str(top_gainer.symbol[3])+'  -  '+str(top_gainer.highPrice[3])

# Reading Data using Yahoo API and CSV Files


In [ ]:
root = tk.Tk()
root.configure(background='#5DADE2')

def on_keyrelease(event):

    # get text from entry
    value = event.widget.get()
    value = value.strip().lower()

    # get data from test_list
    if value == '':
        data = words
    else:
        data = []
        for item in words:
            if value in item.lower():
                data.append(item)                

    # update data in listbox
    listbox_update(data)

def listbox_update(data):
    # delete previous data
    listbox.delete(0, 'end')

    # sorting data 
    data = sorted(data, key=str.lower)

    # put new data
    for item in data:
        listbox.insert('end', item)

labelfont = ('Trebuchet MS',12,'bold')
checkfont = ('Trebuchet MS',12)
downloadfont=('Trebuchet MS',12)
displayfont=('Trebuchet MS',15)




entry = tk.Entry(root)
entry.grid(row=1,column=1,sticky=W,pady=0,padx=0)
entry.bind('<KeyRelease>', on_keyrelease)

load_button = Button(root,text='Import Data',command=read_data,bg="#F4D03F")
load_button.config(font=downloadfont)
load_button.grid(row=1, column=2, sticky=S, padx=3,pady=3)

 
listbox = tk.Listbox(root)
listbox.grid(row=2,column=1,sticky=W,padx=0,pady=0)
#listbox.bind('<Double-Button-1>', on_select)
listbox.bind('<<ListboxSelect>>', load_data)
listbox_update(words)

label1= Label(root, text='',bg = "#5DADE2",fg = "yellow")
label1.config(font=displayfont)
label1.grid(row=1,column=3,sticky=NW, pady=0,padx=0)

tree = ttk.Treeview(root,selectmode="extended",height=10)
tree.config(columns=['Open','high','low','close'])
tree.column("#0",minwidth=0,width=100, stretch=NO)

tree.heading('#1', text='open')
tree.column("#1",minwidth=0,width=100, stretch=NO)

tree.heading('#2', text='high')
tree.column("#2",minwidth=0,width=100, stretch=NO)

tree.heading('#3', text='low')
tree.column("#3",minwidth=0,width=100, stretch=NO)

tree.heading('#4', text='close')
tree.column("#4",minwidth=0,width=100, stretch=NO)

tree.grid(row=2, column=2, sticky=SE,pady=10,padx=10)

# Creating new tree for displaying predicted values

label2= Label(root, text='',bg = "#5DADE2",fg = "yellow")
label2.config(font=displayfont)
label2.grid(row=7,column=2,sticky=N, pady=0,padx=0)
label3= Label(root, text='',bg = "#5DADE2",fg = "yellow")
label3.config(font=displayfont)
label3.grid(row=8,column=2,sticky=N, pady=0,padx=0)
label8= Label(root, text='',bg = "#5DADE2",fg = "yellow")
label8.config(font=displayfont)
label8.grid(row=11,column=2,sticky=N, pady=0,padx=0)
label4= Label(root, text='',bg = "#5DADE2",fg = "yellow")
label4.config(font=displayfont)
label4.grid(row=12,column=2,sticky=N, pady=0,padx=0)
label5= Label(root, text='',bg = "#5DADE2",fg = "yellow")
label5.config(font=displayfont)
label5.grid(row=13,column=2,sticky=N, pady=0,padx=0)
label6= Label(root, text='',bg = "#5DADE2",fg = "yellow")
label6.config(font=displayfont)
label6.grid(row=14,column=2,sticky=N, pady=0,padx=0)
label7= Label(root, text='',bg = "#5DADE2",fg = "yellow")
label7.config(font=displayfont)
label7.grid(row=15,column=2,sticky=N, pady=0,padx=0)





# #importing tree
tree2 = ttk.Treeview(root,selectmode="extended",height=5)
tree2.config(columns=['Predicted','Live'])
tree2.column("#0",minwidth=0,width=100, stretch=NO)

tree2.heading('#1', text='Predicted')
tree2.column("#1",minwidth=0,width=100, stretch=NO)

tree2.heading('#2', text='Live')
tree2.column("#2",minwidth=0,width=100, stretch=NO)



tree2.grid(row=10, column=2, sticky=S,pady=0,padx=0)



train_button = Button(root,text='Train the model',command=train_fun,bg="#F4D03F")
train_button.config(font=downloadfont)
train_button.grid(row=6, column=2, sticky=N, padx=0,pady=0)



root.mainloop()

(event) previous: .ipynb_checkpoints
20MICRONS_data.csv


/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



# Yahoo API Sample code [REF]

In [ ]:
#import external pandas_datareader library with alias of web
import pandas_datareader as web
 
#import datetime internal datetime module
#datetime is a Python module
import datetime
 
#datetime.datetime is a data type within the datetime module
start = datetime.datetime(2018, 2, 19)
end = datetime.datetime(2018, 10, 22)
 
#DataReader method name is case sensitive
data = web.DataReader("20MICRONS.NS",'yahoo', start, end)
 
#invoke to_csv for df dataframe object from 
#DataReader method in the pandas_datareader library
 
#..\first_yahoo_prices_to_csv_demo.csv must not
#be open in another app, such as Excel
 
data.to_csv('first_yahoo_prices_volumes_to_csv_demo.csv')